In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:90%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:10pt;}
div.text_cell_render.rendered_html{font-size:11pt;}
div.output {font-size:10pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:10pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:10pt;padding:5px;}
table.dataframe{font-size:10px;}
</style>
"""))

<b><font size="6" color="red">Ch 09. Red Wine 품질 등급 예측 실습</font><b>

* 1. 데이터셋 확보 & 전처리
독립변수와 타겟변수 분리 -> 독립변수 스케일조정(StandardScaler)
→ 타겟변수원핫인코딩(get_dummies) -> 훈련셋과 테스트셋분리(train_test_split)
* 2. 모델 구성(입력11, 출력?)
* 3. 모델학습과정 설정(다중분류로 설정)
* 4. 모델 학습(callbacks 이용)
* 5. 모델평가 - 그래프, 평가(테스셋), 교차표
* 6. 모델 저장/ 사용

In [2]:
import numpy as np
import pandas as pd # read_csv, get_dummies
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.layers import Input, Dense, Dropout, LeakyReLU
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# 1. 데이터셋 확보 & 전처리

```고정 산도(fixed acidity)
휘발성 산도(volatile acidity)
구연산(citric acid)
잔여 당분(residual sugar)
염화물(chlorides)
유리 아황산염(free sulfur dioxide)
총 아황산염(total sulfur dioxide)
밀도(density)
pH
황산염(sulphates)
알코올(alcohol)
```

In [4]:
#데이터 읽어오기
redwine = pd.read_csv('./data/winequality-red.csv', sep=';')
redwine.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [5]:
# 타겟변수의 균형
redwine['quality'].value_counts()

5    681
6    638
7    199
4     53
8     18
3     10
Name: quality, dtype: int64

In [9]:
# 독립변수, 타깃변수 분리
X_redwine = redwine.iloc[:, :-1].values # == to_numpy
y_redwine = redwine.iloc[:, -1]
X_redwine.shape, y_redwine.shape

((1599, 11), (1599,))

In [13]:
# 독립변수 X의 스케일 조정
scaler = StandardScaler()
scaled_X = scaler.fit_transform(X_redwine)
# 다중 분류를 위한 타깃변수의 원핫인코딩
Y_redwine = pd.get_dummies(y_redwine).to_numpy()

In [14]:
# 독립변수: scaled_X_redwine & 타깃변수 : Y_redwine
scaled_X.shape, Y_redwine.shape

((1599, 11), (1599, 6))